In [25]:
from dotenv import load_dotenv
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Load environment variables
load_dotenv()

True

In [37]:
# Get API key
google_api_key = os.getenv("GOOGLE_API_KEY")
# google_api_key

In [29]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

In [31]:
video_id = "Gfr50f6ZBvo" # only the ID, not full URL
try:
    # If you don’t care which language, this returns the “best” one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    # print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")

In [34]:
spilter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = spilter.create_documents([transcript])

In [36]:
len(chunks)

168

In [40]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
vector_store = FAISS.from_documents(chunks,embeddings)

In [42]:
# vector_store.index_to_docstore_id

In [43]:
retriever = vector_store.as_retriever(search_type='similarity',search_kwargs={'k':4})

In [45]:
retriever.invoke('who is lex fridman')

[Document(id='505c917c-c1bd-4a36-bf04-c7e22572a5e5', metadata={}, page_content="the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal qu

In [46]:
from langchain_google_genai import GoogleGenerativeAI

In [64]:
llm = GoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)
# Initialize the Gemini model correctly
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",  # Use "gemini-pro" or "gemini-pro-vision"
#     google_api_key=os.getenv("GOOGLE_API_KEY"),
#     temperature=0.5
# )

In [65]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [66]:
question          = "is the topic of nuclear fusion discussed in this video? if yes then what was discussed"
retrieved_docs    = retriever.invoke(question)

In [67]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"in this case in fusion we we collaborated with epfl in switzerland the swiss technical institute who are amazing they have a test reactor that they were willing to let us use which you know i double checked with the team we were going to use carefully and safely i was impressed they managed to persuade them to let us use it and um and it's a it's an amazing test reactor they have there and they try all sorts of pretty crazy experiments on it and um the the the what we tend to look at is if we go into a new domain like fusion what are all the bottleneck problems uh like thinking from first principles you know what are all the bottleneck problems that are still stopping fusion working today and then we look at we you know we get a fusion expert to tell us and then we look at those bottlenecks and we look at the ones which ones are amenable to our ai methods today yes right and and and then and would be interesting from a research perspective from our point of view from an ai point of\n\

In [68]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [70]:
answer = llm.invoke(final_prompt)
print(answer)

Yes, nuclear fusion is discussed in the video.

Here's what was discussed:

*   The speaker collaborated with EPFL in Switzerland, a Swiss technical institute with a test reactor, to work on fusion.
*   They identify bottleneck problems in fusion that are amenable to AI methods.
*   They use AI to control high-temperature plasmas by controlling the magnetic field, aiming to react in real-time to the plasma's behavior.
*   They held the plasma in specific shapes using AI, publishing their work in a Nature paper.
*   The speaker believes AI can help accelerate progress in energy and climate, with fusion being one area.
*   Fusion has challenges in physics, material science, and engineering.

